<a href="https://colab.research.google.com/github/ambarishg/open-source-llms/blob/main/RAG_ZEPHYR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install qdrant-client sentence-transformers accelerate

In [2]:
from transformers import BitsAndBytesConfig
import torch

In [3]:
# Qdrant server URL
URL ="20.119.32.168"
# Qdrant dimension of the collection
DIMENSION = 384
# Qdrant collection name
COLLECTION_NAME = "SEARCH_ENGINE"
METRIC_NAME ="COSINE"
MODEL_NAME="all-MiniLM-L6-v2"

In [4]:
import qdrant_client as qc
import qdrant_client.http.models as qmodels
from qdrant_client.http.models import *

In [5]:
from sentence_transformers import SentenceTransformer

In [6]:
import torch
from transformers import pipeline

In [7]:
user_input = "Describe Platyhelminthes"
query_filter =  None

In [8]:
def get_qdrant_client():
    client = qc.QdrantClient(url=URL)
    METRIC = qmodels.Distance.COSINE
    return client

In [9]:
client = get_qdrant_client()

In [10]:
client

In [11]:
def load_model():
    return SentenceTransformer(MODEL_NAME)

In [12]:
model = load_model()
xq = model.encode(user_input,convert_to_tensor=True)

In [13]:
xq

tensor([-1.1949e-02,  7.3089e-02,  3.0627e-03, -1.5451e-03, -5.1068e-02,
         1.5868e-02,  4.4833e-03,  5.5065e-02, -3.6888e-02, -3.5290e-02,
         6.5472e-03, -1.0810e-01,  6.8254e-02,  3.8157e-02, -7.4797e-02,
         1.8154e-02, -8.7613e-03, -4.9936e-02,  5.8443e-04, -2.5256e-02,
         4.7975e-02,  4.8023e-02,  9.4440e-03, -1.1230e-02, -7.0084e-02,
         2.9223e-02, -7.1057e-02,  2.5630e-02,  2.6364e-02, -5.6030e-02,
        -3.0597e-03, -2.0097e-02, -1.7338e-02,  1.1662e-02,  4.1360e-02,
         4.7449e-02,  5.9810e-02,  5.9318e-02, -8.5401e-03,  4.5458e-02,
         2.5633e-02,  8.2683e-02, -6.4137e-02,  3.6973e-02, -7.0319e-02,
        -1.0289e-01, -2.8695e-02, -1.1400e-03, -4.8947e-02,  4.9783e-03,
        -1.1375e-02, -7.3111e-02, -5.6699e-02,  1.0431e-01,  3.7963e-02,
         1.2424e-02, -6.7410e-02, -6.1781e-03, -1.4287e-02, -5.5694e-02,
        -4.2496e-02,  4.4448e-02,  9.6283e-02,  5.0947e-03, -6.0253e-02,
         1.7857e-02, -6.6891e-02, -1.8490e-02, -1.5

In [14]:
search_result = client.search(collection_name=COLLECTION_NAME,
                                    query_vector=xq.tolist(),
                                    query_filter=query_filter,
                                    limit=5)

In [15]:
search_result

[ScoredPoint(id='609555e6-6f9b-42cb-9bdc-b58c58c2d775', version=0, score=0.7078419, payload={'filename': 'Chap04AnimalKingdom.pdf', 'token': '9 Examples of Platyhelminthes : (a) Tape worm (b) Liver fluke\nRationalised 2023-24\nANIMAL KINGDOM 4343\n4.2.5 Phylum – Aschelminthes\nThe body of the aschelminthes is circular in\ncross-section, hence, the name roundworms\n(Figure 4.10). They may be freeliving, aquatic\nand terrestrial or parasitic in plants and animals.\nRoundworms have organ-system level of body\norganisation. They are bilaterally symmetrical,\ntriploblastic and pseudocoelomate animals', 'type': 'pdf'}, vector=None, shard_key=None),
 ScoredPoint(id='72be533c-020e-4590-8609-083ad4c810ef', version=0, score=0.6169232, payload={'filename': 'Chap04AnimalKingdom.pdf', 'token': ' Reproduction takes place only by sexual\nmeans. Fertilisation is external with indirect development.\nExamples: Pleurobrachia  and Ctenoplana .\n4.2.4 Phylum – Platyhelminthes\nThey have dorso-ventrally fla

In [16]:
contexts =""
for result in search_result:
    contexts +=  result.payload['token']+"\n---\n"

In [17]:
contexts

'9 Examples of Platyhelminthes : (a) Tape worm (b) Liver fluke\nRationalised 2023-24\nANIMAL KINGDOM 4343\n4.2.5 Phylum – Aschelminthes\nThe body of the aschelminthes is circular in\ncross-section, hence, the name roundworms\n(Figure 4.10). They may be freeliving, aquatic\nand terrestrial or parasitic in plants and animals.\nRoundworms have organ-system level of body\norganisation. They are bilaterally symmetrical,\ntriploblastic and pseudocoelomate animals\n---\n Reproduction takes place only by sexual\nmeans. Fertilisation is external with indirect development.\nExamples: Pleurobrachia  and Ctenoplana .\n4.2.4 Phylum – Platyhelminthes\nThey have dorso-ventrally flattened body, hence are called\nflatworms (Figure 4.9). These are mostly endoparasites found\nin animals including human beings. Flatworms are bilaterally\nsymmetrical, triploblastic and acoelomate animals with organ\nlevel of organisation. Hooks and suckers are present in the\nparasitic forms\n---\n\nPorifera includes multi

In [18]:
# Install transformers from source - only needed for versions <= v4.34
# pip install git+https://github.com/huggingface/transformers.git
# pip install accelerate

pipe = pipeline("text-generation",
                model="HuggingFaceH4/zephyr-7b-beta",
                torch_dtype=torch.bfloat16,
                device_map="auto")


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [19]:

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": contexts},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
9 Examples of Platyhelminthes : (a) Tape worm (b) Liver fluke
Rationalised 2023-24
ANIMAL KINGDOM 4343
4.2.5 Phylum – Aschelminthes
The body of the aschelminthes is circular in
cross-section, hence, the name roundworms
(Figure 4.10). They may be freeliving, aquatic
and terrestrial or parasitic in plants and animals.
Roundworms have organ-system level of body
organisation. They are bilaterally symmetrical,
triploblastic and pseudocoelomate animals
---
 Reproduction takes place only by sexual
means. Fertilisation is external with indirect development.
Examples: Pleurobrachia  and Ctenoplana .
4.2.4 Phylum – Platyhelminthes
They have dorso-ventrally flattened body, hence are called
flatworms (Figure 4.9). These are mostly endoparasites found
in animals including human beings. Flatworms are bilaterally
symmetrical, triploblastic and acoelomate animals with organ
level of organisation. Hooks and 

In [20]:
response2 = outputs[0]["generated_text"]

In [21]:
response2.split("<|assistant|>")[-1]

"\nAvast me hearties, you be seekin' knowledge 'bout the animal kingdom!\n\nWell, let me tell ye 'bout the Platyhelminthes, a phylum o' flatworms with a dorso-ventrally flattened body. They're mostly parasites found in animals, including humans, and have acoelomate bodies with organ-level organization.\n\nTheir digestive system is quite unique, as it has a single opening that serves as both mouth and anus, making it incomplete. In contrast, a complete digestive system has two openings, a mouth, and an anus.\n\nAs for the other phyla, Porifera are multicellular animals with flagellated choanocytes and cellular-level organization. Coelenterates have tentacles and cnidoblasts and can be found in water, either sessile or free-floating. Ctenophores are marine animals with comb plates.\n\nThe Platyhelminthes, as well as Aschelminthes (pseudocoelomate roundworms), Annelids (segmented animals with true"